In [2]:
import sys
sys.path.append("../")
from pyproj import CRS
import numpy as np
import sys
from osgeo import gdal
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates as mdates
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
from datetime import datetime
from time import time
from collections import Counter
import pandas as pd #for data analysing
import openpyxl
import os



In [4]:
NDSI_SON_beforSL_b = xr.open_dataset('NDSI_DJF_beforSL_b_nc.nc')
NDSI_SON_beforSL_a = xr.open_dataset('NDSI_DJF_beforSL_a_nc.nc') 

In [6]:
NDSI_SON_beforSL = xr.concat([NDSI_SON_beforSL_a, NDSI_SON_beforSL_b], dim='longitude')

####j'attribue un CRS à mon dataset
crs = CRS.from_string('EPSG:4326')
summer_mask.attrs["crs"] = crs.to_proj4()

In [9]:
lat = NDSI_SON_beforSL['latitude'].values
lon = NDSI_SON_beforSL['longitude'].values

summer_mask = xr.DataArray(summer_mask.layer.values, coords={'latitude': lat,'longitude':lon}, dims=['latitude','longitude'])
NDSI_SON_beforSL['Summer_mask'] = summer_mask
####on change en entier le DEM on a pas besoisn de virgule
#NDSI_SON['DEM'] = NDSI_SON.DEM.astype('int16')

In [12]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_cloudpercent_b(array):
    cloud = np.sum(array == 2)
    #print(cloud)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    percentage_cloud_b = round(((float(cloud) / float(tot_pix)) * 100),2)
    return percentage_cloud_b

In [13]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_cloudpercent_a(array_copy):
    cloud = np.sum(array_copy  == 2)
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    cloud_percent = round((float(cloud) / float(tot_pix)) * 100, 2)
    return cloud_percent

In [14]:
def step2_gapfilling_tot(array,array_copy):
    cloudpercent_b = compute_cloudpercent_b(array)
    cloudpercent_a = compute_cloudpercent_a(array_copy)
    step2_gf_tot = round(((cloudpercent_b)-(cloudpercent_a)),2)
    #step3_gf_tot = ((cloudpercent_b)-(cloudpercent_a))
    return step2_gf_tot

In [15]:
def compute_snowpercent_b(array):
    snow = np.count_nonzero((array == 1) | (array == 100))
    #print(snow)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    snow_percent_b = round(((float(snow) / float(tot_pix)) * 100),2)
    return snow_percent_b

In [16]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_snowpercent_a(array_copy):
    snow = np.count_nonzero((array_copy == 1) | (array_copy == 100))
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    snow_percent = round((float(snow) / float(tot_pix)) * 100, 2)
    return snow_percent

In [17]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_landpercent_b(array):
    land = np.count_nonzero((array == 0) | (array == 200))
    #print(land)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    land_percent_b = round((float(land) / float(tot_pix)) * 100,2)
    return land_percent_b

In [18]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_landpercent_a(array_copy):
    land = np.count_nonzero((array_copy == 0) | (array_copy == 200))
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    land_percent = round((float(land) / float(tot_pix)) * 100, 2)
    return land_percent

In [19]:
cloud_percent_b_list = []
cloud_percent_a_list = []
step2_gapfilling_tot_list = []
snow_percent_b_list = []
snow_percent_a_list = []
land_percent_b_list = []
land_percent_a_list = []
time_list = []

In [21]:
data = {
    'Time': time_list,
    'Cloud_Percent_Before': cloud_percent_b_list,
    'Cloud_Percent_After': cloud_percent_a_list,
    'Cloud_Reduction': step2_gapfilling_tot_list,
    'Snow_Percent_Before': snow_percent_b_list,
    'Snow_Percent_After': snow_percent_a_list,
    'Land_Percent_Before': land_percent_b_list,
    'Land_Percent_After': land_percent_a_list
}

# Créer un DataFrame à partir du dictionnaire
df_step2 = pd.DataFrame(data)
filename = f'df_summer_mask_DJF.xlsx'
filepath = os.path.join('/home/charlottepoussin/Excels', filename)
df_step2.to_excel(filepath, index=False)